In [76]:
import pandas as pd 
from datetime import datetime
from forex_python.converter import CurrencyRates
import re


In [3]:
Expedia_hotels = pd.read_csv('Expedia_hotels.csv')

C:\Users\yarin\AppData\Local\Temp\ipykernel_5148\1586826330.py:1: DtypeWarning: Columns (13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Expedia_hotels = pd.read_csv('Expedia_hotels.csv')


In [80]:
df = Expedia_hotels.copy()

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Snapshot                36000 non-null  object 
 1   Index                   36000 non-null  int64  
 2   Hotel Name              35980 non-null  object 
 3   TTT                     35980 non-null  float64
 4   LOS                     35980 non-null  float64
 5   Grade                   35468 non-null  float64
 6   Num of Reviews          35468 non-null  object 
 7   Curr Price              35980 non-null  object 
 8   Original Price          15342 non-null  object 
 9   Percentage of discount  35980 non-null  float64
 10  Distance from center    35980 non-null  float64
 11  Type of room            35980 non-null  float64
 12  Location grade          35980 non-null  float64
 13  Is refundable           35980 non-null  object 
 14  Late payment            35980 non-null

In [82]:
df['Snapshot'] = df['Snapshot'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').date())
df = df.loc[df['Index'] < 100]
df['Index'] = df['Index'] + 1
df['TTT'] = df['TTT'].astype('int64')
df['LOS'] = df['LOS'].astype('int64')


In [84]:
df['Num of Reviews'] = df['Num of Reviews'].apply(lambda x: int(re.sub("[^0-9]", "", x)) if not pd.isnull(x) else x)

In [85]:
df['Num of Reviews']

0        1296.0
1        1002.0
2         305.0
3        3123.0
4        1465.0
          ...  
35975    1000.0
35976    1000.0
35977    1001.0
35978     999.0
35979    1106.0
Name: Num of Reviews, Length: 30000, dtype: float64

In [87]:
df['Grade'].describe()

count    29494.000000
mean         8.404733
std          0.645410
min          5.600000
25%          8.000000
50%          8.600000
75%          8.800000
max         10.000000
Name: Grade, dtype: float64

In [88]:
df.loc[pd.isnull(df['Grade'])]['Hotel Name'].value_counts()

Latisha luxury place                148
Moxy Brooklyn Willamsburg           115
The FIDI Hotel                      111
The Ritz-Carlton New York, NoMad    107
The Flat NYC                         23
Latisha house                         2
Name: Hotel Name, dtype: int64

In [89]:
df.loc[pd.isnull(df['Num of Reviews'])]['Hotel Name'].value_counts()

Latisha luxury place                148
Moxy Brooklyn Willamsburg           115
The FIDI Hotel                      111
The Ritz-Carlton New York, NoMad    107
The Flat NYC                         23
Latisha house                         2
Name: Hotel Name, dtype: int64

In [90]:
df['Curr Price'] = df['Curr Price'].apply(lambda x: x.split('€')[1])

In [91]:
df['Original Price'] = df['Original Price'].apply(lambda x: x.split('€')[1] if not pd.isnull(x) else x)

In [92]:
df['Original Price']

0          360
1          NaN
2          493
3          NaN
4          NaN
         ...  
35975      NaN
35976    1,958
35977    2,150
35978      NaN
35979      NaN
Name: Original Price, Length: 30000, dtype: object

In [93]:
df['Original Price'] = df.apply(lambda x: x['Curr Price']  if pd.isnull(x['Original Price']) else x['Original Price'], axis = 1)
df['Original Price']

0          360
1          217
2          493
3          328
4          428
         ...  
35975    1,656
35976    1,958
35977    2,150
35978    1,926
35979    1,461
Name: Original Price, Length: 30000, dtype: object

In [94]:
df['Original Price'] = df['Original Price'].apply(lambda x: int(x.replace(',','')))
df['Curr Price'] = df['Curr Price'].apply(lambda x: int(x.replace(',','')))

In [95]:
df['Original Price']

0         360
1         217
2         493
3         328
4         428
         ... 
35975    1656
35976    1958
35977    2150
35978    1926
35979    1461
Name: Original Price, Length: 30000, dtype: int64

In [96]:
c = CurrencyRates()
x = df['Snapshot'].unique()
curr_dict = {}
for date in x:
    curr_dict[date] = c.get_rate('EUR','USD', date) 



In [97]:
curr_dict[df['Snapshot'].iloc[0]]

1.0646

In [98]:
df['Original Price']

0         360
1         217
2         493
3         328
4         428
         ... 
35975    1656
35976    1958
35977    2150
35978    1926
35979    1461
Name: Original Price, Length: 30000, dtype: int64

In [99]:
df['Original Price'] = df.apply(lambda x: (x['Original Price']*curr_dict[x['Snapshot']])/x['LOS']  , axis = 1)
df['Curr Price'] = df.apply(lambda x: (x['Curr Price']*curr_dict[x['Snapshot']])/x['LOS']  , axis = 1)


In [100]:
df['Original Price'] = df['Original Price'].apply(lambda x: round(x,1))
df['Curr Price'] = df['Curr Price'].apply(lambda x: round(x,1))

In [101]:
df[['Original Price','Curr Price']]

,Original Price,Curr Price
0,383.3,296.0
1,231.0,231.0
2,524.8,478.0
3,349.2,349.2
4,455.6,455.6
...,...,...
35975,353.2,353.2
35976,417.6,351.3
35977,458.6,412.9
35978,410.8,410.8


In [102]:
df.drop(['Percentage of discount', 'Distance from center', 'Type of room',
       'Location grade'], axis = 1, inplace=True)

In [105]:
df[['Is refundable','Late payment', 'Option Member']] = df[['Is refundable','Late payment', 'Option Member']].astype('int64')

In [108]:
df['Late payment'].value_counts()

0    29740
1      260
Name: Late payment, dtype: int64